In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data_processing/cleaned-data.csv')

# Binary Features

There are ten binary features in the raw dataset. I will convert these from their descriptive labels to binary `(1, 0)` values for descriptive computational analysis (sum, count, looping) and machine learning. The columns that I will transform are the following:

column | description | current values | transformed values
-------|-------------|----------------|-------------------
school | the school attended by the student | MS and GP | 0 (MS) and 1 (GP)
sex | the student's gender | M and F | 0 (M) and 1 (F)
address | an urban or rural address | U and R | 0 (U) and 1 (R)
famsize | <=3 or >3 in the family | LE3 and GT3 | 0 (LE3) and 1 (GT3)
Pstatus | separated or together parents | A and T | 0 (A) and 1 (T)
schoolsup | learning after school hours | no and yes | 0 (no) and 1 (yes)
famsup | family support at home | no and yes | 0 (no) and 1 (yes)
paid | paid tutoring | no and yes | 0 (no) and 1 (yes)
activities | extracurricular activities | no and yes | 0 (no) and 1 (yes)
nursery | attended nursery school | no and yes | 0 (no) and 1 (yes)
higher | planning for higher education | no and yes | 0 (no) and 1 (yes)
internet | internet access at home | no and yes | 0 (no) and 1 (yes)
romantic | in a romantic relationship | no and yes | 0 (no) and 1 (yes)

# Correlated Features

`G1` and `G2` are described as period one and two grades. In my experience, these features would correspond to trimester one (G1) and trimester two (G2) grades which lead up to the final grade (G3) for the year. These two features —period one and two grades (G1 & G2)— are highly correlated to the responce variable (G3) because they represent the same grading process and performance metric that we are trying to predict. However, there is a powerful binary feature that can be engineered from these two features (G1 & G2) that gets to the source of improved learning perforance, resiliance, and grit; improved grades from period one to period two.

I will engineer a new binary feature that is `0` —for students who had grade depreciation or maintained the same grade from period one to period two— or `1` —for students who improved their grade from period one to period two— and use this new feature to perform statistical analysis and machine learning.

    improved = 0, if G1 - G2 >= 0
    improved = 1, if G1 - G2 < 0